## About Kosarak
- Distinct element: 606770
- Number of incomming element: 990002
- Top-10 frequent element and count: 

| element | frquent |
| :----- | :----- |
| 3 | 36133 |
| 6 | 30225 |
| 6 3 | 21139 |
| 11 6 | 12769 |
| 1 | 9766 |
| 11 6 3 | 9083 |
| 1 3 | 6002 |
| 11 | 5374 |
| 1 6 | 5046 |
| 4 | 4290 |


In [ ]:
# 20210316 pseudo code

import sys
import time
import spookyhash
import hyperloglog

def UpdateSk(e,c,width,depth):
    r=hash1(e)% depth
    ID=hash2(hash1(e))
        # compute hash value of e as ID
    found=False
        # found= True if e is in Sk[r]
    index=0
    Sk[r][0].count+=1    # total count+1
    for i in range(len(Sk[r])):
        # checks whether e is in SK[h1(e)]
        if ID == Sk[r][i].ID:
            found=True
            index=i
    if found:  # e is in Sk[r]
        Sk[r][index].count+=1 
    else:  # e is not in Sk[r]
        if len(Sk[r])<width:
            check=random.choice([0,1],size=1,p=[r/w,1-r/w])
            # append e with prob(1-r/w)
            if check:
                # put e into Sk[r]
                Sk[r].append(ID,1)
                index=len(Sk[r])-1
                found=True
        else:
            # put e into other=total count-sum(Sk[r][i].count)
            # estimate distinct element in other
            #　Sk[r][0].distinct=update(Sk[r][0].distinct,hash3(e))
            hll.add(e)
    if found:
        if Sk[r][index].count>cmax:
            emax=e
            cmax=Sk[r][index].count
    else:
        avg=(Sk[r][0].count-sum(Sk[r][i].count))/Sk[r][0].distinct
        if avg >cmax:
            emax=e
            cmax=avg
            

# main
import sys
import time

if __name__=='main':
    start=time.time()
    size=32
    Top=[size]
    Sk=[]
    item_count=100
    depth=1024
    width=4
    emax,cmax='',0
    with open("kosarak.dat",'r') as file:
        while item:
            element=file.readline().strip('\n')
            if not element:
                break
            else:
                item_count-=1
                if len(Top)==0:
                    Top.append([element,1])
                else:
                    # if e in Top
                    for i in range(len(Top)):
                        if element==Top[i][0]:
                            Top[i][1]+=1
                            found=True
                            #print("found in itemlist[{}]".format(i))
                            break
                        else:
                            found=False
                    if found:
                        pass
                    else:
                        if len(Top)<size:
                            Top.append([element,1])                        
                        else:
                            UpdateSk(element,1,width,depth)
                if cmax>Top[-1].count:
                    # bring back
                    tempe=Top[-1].ID
                    tempc=Top[-1].count
                    Top[-1].ID=emax
                    Top[-1].count=cmax
                    UpdateSk(tempe,tempc,width,depth)
    end=time.time()

print(Top)
print("Top-{} with size {} bytes.".format(len(itemlist),sys.getsizeof(itemlist)))
print("Execution time:{} seconds.".format(str(end-start)))


## Hyperloglog

In [ ]:
# https://github.com/svpcom/hyperloglog
import hyperloglog
import sys
import time

start=time.time()
datapath='../CountMinsketch/kosarak.dat'

hll = hyperloglog.HyperLogLog(0.01)  # accept 1% counting error

item_count=1000
with open(datapath,'r') as file:
    while True:
        element=file.readline().strip('\n')
        if not element:
            break
        else:
            # item_count-=1
            hll.add(element)
end=time.time()        
print(len(hll),type(hll),sys.getsizeof(hll))
print("Execution time:{} seconds.".format(str(end-start)))

## Space Saving full test
- Execution time: 270.0425057411194 seconds

In [8]:
# space saving kosarak, top-1024 elements, full test
# Execution time: 270.0425057411194 seconds

import sys
import time
import spookyhash
import mmh3
import hyperloglog
import operator

class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def sub_count(self,count=1):
        self.count-=count
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self):
        super().__init__()
        self.distinct = hyperloglog.HyperLogLog(0.01)
    def __str__(self):
        return 'total count: {}, distinct element: {}'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return "'{}', count: {}".format(self.ID,self.count)

start=time.time()
datapath='..\CountMinsketch\kosarak.dat'
size=1024
Top=[]


with open(datapath,'r') as file:
    while True:
        element=file.readline().strip('\n')
        # print(element)
        if not element:
            break
        else:
            if len(Top)==0:
                Top.append(Tail(element,1))
            else:
                found=False
                for i in range(len(Top)):
                    if Top[i].ID==element:
                        found=True
                        Top[i].add_count()
                        break
                if not found:
                    if len(Top)<size:
                        Top.append(Tail(element,1))
                    else:
                        Top[-1].ID=element
                        Top[-1].add_count()
                # Top= sorted(Top, key=operator.attrgetter('count'),reverse=True)
                # non-inplace sort
                Top.sort(key=operator.attrgetter('count'),reverse=True)
                    # inplce sort
end=time.time()
print('Execution time: {} seconds'.format(str(end-start)))
print('Top-100 frequent item: {}'.format(Top[:100]))

Execution time: 229.3381245136261 seconds
Top-100 frequent item: ['3', count: 36133, '6', count: 30225, '6 3', count: 21139, '11 6', count: 12769, '1', count: 9766, '11 6 3', count: 9083, '1 3', count: 6002, '11', count: 5374, '1 6', count: 5046, '4', count: 4290, '11 3', count: 3756, '55', count: 3561, '1 6 3', count: 3510, '4 3', count: 3000, '55 3', count: 2377, '11 1 6', count: 2375, '4 6', count: 2341, '4 6 3', count: 1758, '11 1 6 3', count: 1753, '6 55', count: 1749, '294', count: 1653, '2', count: 1601, '6 55 3', count: 1375, '490', count: 1226, '27', count: 1168, '294 3', count: 1162, '2 3', count: 1152, '7', count: 1100, '218', count: 1056, '11 4 6', count: 954, '64', count: 915, '6 294', count: 893, '6 2', count: 882, '11 1', count: 880, '91', count: 866, '73', count: 853, '490 3', count: 846, '11 4 6 3', count: 828, '6 294 3', count: 823, '4 1', count: 821, '6 2 3', count: 821, '3 73', count: 820, '27 6 3', count: 820, '27 3', count: 819, '6 7', count: 819, '11 6 55', count

## My algorithm

In [7]:
# kosarak, top-512, sketch=4*128, 0.01 error rate of hyperloglog

import sys
import time
import spookyhash
import mmh3
import hyperloglog
import operator
import unittest
from numpy import random

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================



# ========================== Update Sketch==========================
def UpdateSk(element,width,depth):
    #　width+=1
    e_max=get_emax()
    # print("first read e_max in UpdateSK, id(e_max):{}".format(id(e_max)))
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
        # input of spooky hash: binary, encoding is parameter of bytes()
        # output of spooky hash: unsigned- 32 bit int
    hash2=mmh3.hash(str(hash1), signed=False)
        # input of mmh3 is str, output unsigned- 32 bit int
    row=hash1 % depth
        # index of row in SK[row]
    ID=hash2 % ((width*3)//2)
        # hash-value(ID) of e
        # range: 2^k *3/2= 3*(2^k-1)
    match=False
        # match= True if e is in Sk[row]
    index=0
    item=Tail(ID,element.count)
    
    # print('UpdateSk:{}->ID {}'.format(element,ID))
    # print("hash1: {}".format(hash1))
    # print("hash2: {}".format(hash2))
    # print("match row: {}".format(row))
    # print("SK[{}]:{}".format(row,Sk[row]))
    # print("len(Sk[{}]:{})".format(row,len(Sk[row])))
    
    Sk[row][0].add_count(item.count)
        # total count+=count
    if len(Sk[row])==1:
        # Head only, append e directly
        Sk[row].append(item)
        Sk[row][0].maxID=element.ID
        match=True
        index=1
            # e is in Sk[row][1]
        # print('Sk[{}] when len(Sk[{}])==1:\n\t{}'.format(row,row,Sk[row]))
        # print("After append:\n{}".format(Sk[row]))
        # print("length of Sk[row]: {}".format(len(Sk[row])))
    elif len(Sk[row])<width:
        put_in=random.choice([0,1],size=1,p=[len(Sk[row])/width,1-len(Sk[row])/width])[0]
            # send into SK[row] with prob. 1-len(SK[row])/width
        for i in range(1,len(Sk[row])):
            # checks whether e is in SK[row]
            if ID == Sk[row][i].ID:
                match=True
                Sk[row][i].add_count(item.count)
                index=i
                '''
                print('{} matches in Sk[{}][{}]:{}'.format(item.ID,row,index,Sk[row][index].ID))
                print('Sk[{}]:\n\t{}'.format(row,Sk[row]))
                '''

                break
            else:
                match=False
        if not match:
            if put_in==1:
                # send e into SK[row]
                # print('put_in={}, send e into SK[{}]'.format(put_in,row))
                Sk[row].append(item)
                match=True
                index=len(Sk[row])-1
                # print('Sk[{}]:\n\t{}\n'.format(row,Sk[row]))
            else:
                Sk[row][0].distinct.add(ID)
                ''' 
                print('put_in={}, send e into other,update distinct'.format(put_in,len(Sk[row][0].distinct)))
                print('Sk[{}]:\n\t {}'.format(row,Sk[row]))
                '''

    elif len(Sk[row])==width:
        # len(Sk[row])=width, put e into other
        Sk[row][0].distinct.add(ID)
    # print("match:{}".format(match))
   
    # 這段會影響e_max
    if match:
        # print("e_max in UpdateSK line 114, id(e_max):{}".format(id(e_max)))
        if Sk[row][index].count>e_max.count:
            e_max.ID=element.ID
            e_max.count=Sk[row][index].count
        # print('line 115 e_max: e_max={}'.format(e_max))
            
    else:
        # avg count of other > e_max.count
        avg=(Sk[row][0].count-sum(Sk[row][i].count for i in range(len(Sk[row]))))/len(Sk[row][0].distinct)
        if avg >e_max.count:
            e_max=Tail(element.ID,Sk[row][index].count)
        # print('line 122 e_max: e_max={}'.format(e_max))
    # print('line 123 e_max:{}'.format(get_emax()))
    # print('')
# ========================== Update Sketch==========================
def get_emax():
    return e_max
# ========================== BringBack=========================
def BringBack(e_min,e_max):
    # print("Top before Bringback:\n\t{}".format(Top))
    e_max=get_emax()
    # print('e_max at first Bringback:{},id(e_max):{}'.format(e_max,id(e_max)))
    temp=Tail(e_min.ID,e_min.count)
    Top[-1].ID=e_max.ID
    Top[-1].count=e_max.count
    Top.sort(key=operator.attrgetter('count'),reverse=True)
    # print('Top after BringBack:\n\t{}'.format(Top))
    DeleteSk(e_max)
        # e_max in Sk[row]
    e_max.ID=''
    e_max.count=0
    # print("e_max after delete:{},id(e_max):{}".format(e_max,id(e_max)))
    UpdateSk(temp,width,depth)
    
    # print("Sk[] after Update {}:\n\t{}".format(e_min,Sk))

# ========================== BringBack=========================
def DeleteSk(element):
    # 刪除e_max，找新的e_max
    # print('Delete start======================================')
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
    ID=hash2 % ((width*3)//2)
    row=hash1 % depth
    index=0
    for i in range(1,len(Sk[row])):
        if Sk[row][i].ID==ID:
            # print("found {} at SK[{}][{}], ID={}".format(element.ID,row,i,Sk[row][i].ID))
            index=i
            # print("pop element:{}".format(Sk[row][index]))
            Sk[row].pop(index)
            break
    # print('Delete Sk for find max of Sk[i] len(Sk):{}'.format(len(Sk)))
    Sk[row][0].count-=element.count
    '''
    print("index:{}".format(index))
    print('After pop:')
    print("\tSk[{}]={}".format(row,Sk[row]))
    print("Top:\n\t{}".format(Top))
    print('Delete Sketch over======================================')
    '''


# ========================== BringBack=========================


start=time.time()
datapath='..\CountMinsketch\kosarak.dat'
depth=4
width=128
size=512

Top=[]
Sk=[[Head(0)] for j in range(depth)]
item_count=1000

e_max=Tail('',0)


with open(datapath,'r') as file:
    while True:
        element=file.readline().strip('\n')
        # print("read element: {}".format(element))
        if not element:
            break
        else:
            # Update_Top(Tail(element,1))
            # item_count-=1
            if len(Top)==0:
                Top.append(Tail(element,1))
            else:
                found=False
                for i in range(len(Top)):
                    if Top[i].ID==element:
                        found=True
                        Top[i].add_count(1)
                        break
                if not found:
                    if len(Top)<size:
                        Top.append(Tail(element,1))
                    else:
                        # print("e_max in main, id(e_max):{}".format(id(e_max)))
                        UpdateSk(Tail(element,1),width,depth)
            Top.sort(key=operator.attrgetter('count'),reverse=True)
            # print("Top:{}".format(Top))
            if e_max.count>Top[-1].count:
                BringBack(Top[-1],e_max)
                # bring back
                # 要去掉SK[row]中的e_max.ID及count的node
                # print('Top after BringBack: \n\t{}'.format(Top))

end=time.time()
print("Top[20]:\n\t{}".format(Top[:20]))
print("Sk:\n\t{}".format(Sk))
print('Execution time: {} seconds'.format(str(end-start)))
print("Total size {} bytes.".format(sys.getsizeof(topk)+sys.getsizeof(Sk)))


Top[20]:
	[[ID: 3, count: 36133], [ID: 6, count: 30225], [ID: 6 3, count: 21139], [ID: 11 6, count: 12769], [ID: 1, count: 9766], [ID: 11 6 3, count: 9083], [ID: 1 3, count: 6002], [ID: 11, count: 5374], [ID: 1 6, count: 5046], [ID: 4, count: 4290], [ID: 11 3, count: 3756], [ID: 55, count: 3561], [ID: 1 6 3, count: 3510], [ID: 4 3, count: 3000], [ID: 55 3, count: 2377], [ID: 11 1 6, count: 2375], [ID: 4 6, count: 2341], [ID: 11 1 6 3, count: 1753], [ID: 4 6 3, count: 1751], [ID: 6 55, count: 1749]]
Sk:
	[[[total count: 182768, distinct: 192], [ID: 78, count: 56], [ID: 113, count: 55], [ID: 71, count: 45], [ID: 79, count: 48], [ID: 128, count: 53], [ID: 60, count: 52], [ID: 127, count: 45], [ID: 4, count: 50], [ID: 96, count: 54], [ID: 36, count: 53], [ID: 31, count: 46], [ID: 21, count: 54], [ID: 156, count: 46], [ID: 148, count: 49], [ID: 112, count: 39], [ID: 147, count: 50], [ID: 77, count: 30], [ID: 15, count: 48], [ID: 117, count: 48], [ID: 129, count: 37], [ID: 14, count: 46], [I

In [10]:
import sys
print(sys.getsizeof(Top)+sys.getsizeof(Sk))

9088


In [4]:
# count distinct element frequency
import pandas as pd 
item_count=1000
num_line=0
count=0
itemdict={}
datapath='..\CountMinsketch\kosarak.dat'

with open(datapath,'r') as file:
    while item_count:
        line=file.readline().strip('\n')
        if not line:
            break
        else:
            item_count-=1
            num_line+=1         
            if line not in itemdict:
                itemdict[line]=1
            else:
                itemdict[line]+=1

df=pd.DataFrame(list(itemdict.items()),columns=['Element', 'Count'])
df=df.reset_index(drop=True)
df=df.sort_values('Count',ascending=False)

print("Distinct element: {}".format(len(itemdict)))
print("Number of incomming element: {}".format(num_line))
print(df.head(20))

df.to_csv("kosarak_distinct_count.csv",index=False)


Distinct element: 829
Number of incomming element: 1000
      Element  Count
17          3     34
111         6     22
94       11 6     19
31        6 3     16
1           1     14
37        1 6      9
271       1 3      9
85     11 6 3      9
86       11 3      7
241        55      5
253        11      4
116       294      4
466     1 6 3      3
262         2      3
312  11 4 6 3      3
72        323      3
171     6 2 3      3
233       4 3      3
64       55 3      3
279         4      3


## About AOL
https://www.michael-noll.com/blog/2006/08/07/aol-research-publishes-500k-user-queries/

- AnonID - an anonymous user ID number.
- Query  - the query issued by the user, case shifted with most punctuation removed.
- QueryTime - the time at which the query was submitted for search.
- ItemRank  - if the user clicked on a search result, the rank of the item on which they clicked is listed.
- ClickURL  - if the user clicked on a search result, the domain portion of 
the URL in the clicked result is listed.
